<a href="https://colab.research.google.com/github/HayatoNabetani/yahoo-chiebukuro-scrapying/blob/main/yahoochiebukuro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [6

In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import math

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
              'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
              'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36',
              'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
              ]
options.add_argument(
    '--user-agent=' + user_agent[random.randrange(0, len(user_agent), 1)])

driver = webdriver.Chrome('chromedriver',options=options)
driver.set_window_size('1200', '1000')
driver.implicitly_wait(10)
url = "https://chiebukuro.yahoo.co.jp/"
driver.get(url)

SPREADSHEET_KEY = ''#ここを必ず変更してください。

'''
最初の設定
'''
inputSearchCount= input('取得したい件数を半角数字で入力してください。(何も入力しなければ100件です。): ')

if inputSearchCount == "":
    inputSearchCount = 100
else:
    inputSearchCount = int(inputSearchCount)

inputSearchName = input('検索ワードを入力してください。')
inputSearchOrder = input('表示順序を入れてください。:')
inputSpreadSheetName = input('反映先のスプレッドシートの名前を入れてください。')


'''
スプレッドシートの設定
'''
SHEET_NAME_REFLECTION = inputSpreadSheetName
workbook = gc.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet(SHEET_NAME_REFLECTION)


searchElem = driver.find_element_by_class_name('SearchBox_searchBox__inputBoxInput__nf3fq')
searchWord = inputSearchName
searchElem.send_keys(searchWord)

searchClickElem = driver.find_element_by_class_name('SearchBox_searchBox__inputButton__2OXXW')
searchClickElem.click()

time.sleep(3)

dropdown = driver.find_element_by_tag_name("select")
select = Select(dropdown)

array = ["関連度順","質問日時の新しい順","質問日時の古い順","更新日時の新しい順","更新日時の古い順","回答数の多い順","回答数の少ない順","閲覧数の多い順","閲覧数の少ない順","お礼の多い順","お礼の少ない順"]
index = array.index(inputSearchOrder)
select.select_by_index(index)

time.sleep(3)

totalSearchElem = driver.find_element_by_class_name('Pagination_pagination__result__1GhXj')
totalSearch = totalSearchElem.find_elements_by_tag_name('span')[0].text
_total = totalSearch.split('約')[1]
total = _total.split('件')[0]
totalNum = int(total.replace(',', ''))
print('検索結果数:', totalNum)

searchCount = 0
if inputSearchCount > totalNum:
    searchCount = totalNum
else:
    searchCount = inputSearchCount

worksheet.update_cell(2,3,searchCount)
worksheet.update_cell(3,3,inputSearchOrder)
'''
paginationはsearchCount/10。
余はsearchCount%10だけする。
'''

page = searchCount // 10
page = page + 1
pagelist = searchCount % 10


for p in range(page):
    if p != (page-1):
        listCount = 10
    else:
        listCount = pagelist
    for i in range(listCount):
        time.sleep(2)
        searchResults = driver.find_elements_by_class_name('ListSearchResults_listSearchResults__listItem__F0427')
        link = searchResults[i].find_element_by_tag_name('a')
        watchCount = searchResults[i].find_element_by_class_name('ListSearchResults_listSearchResults__informationViews__7Ovr2').find_elements_by_tag_name('span')[1].text
        print('閲覧数:', watchCount)
        link.click()
        time.sleep(2)
        #まず、Qをゲット
        questionResult = driver.find_element_by_class_name('ClapLv2QuestionItem_Chie-QuestionItem__Title__1NgPA').text
        print('Q:', questionResult)

        answerResult = ""
        try:
            #ベストアンサーがあれば、ベストアンサーをゲット
            answerResults = driver.find_element_by_class_name('ClapLv3BestAnswer_Chie-BestAnswer__1kJ7F')
            _answerResult = answerResults.find_element_by_class_name('ClapLv2AnswerItem_Chie-AnswerItem__ItemText__3yFYH')
            answerResult = _answerResult.find_element_by_tag_name('h2').text
        except:
            print("ベストアンサーなかったよ")
            try:
                #無ければ、回答の最初ゲット
                answerResults = driver.find_element_by_class_name('ClapLv3AnswerList_Chie-AnswerList__Answers__2mp3q')
                _answerResult = answerResults.find_elements_by_class_name('ClapLv3AnswerList_Chie-AnswerList__Item__2PxD4')[0]
                answerResult = _answerResult.find_element_by_class_name('ClapLv1TextBlock_Chie-TextBlock__3X4V5').text
            except:
                #回答が無ければ、空文字のまま
                print("回答がまだないよ")
        finally:
            print('A:', answerResult)
            data = [watchCount, questionResult, answerResult]
            worksheet.append_row(data)

        print("-----------------")
        driver.back()
    pagenations = driver.find_elements_by_class_name('Pagination_pagination__controllerButton__19O9w')
    nextpage = pagenations[-1]
    nextpage.click()
    
driver.close()
driver.quit()


取得したい件数を半角数字で入力してください。(何も入力しなければ100件です。): 25
検索ワードを入力してください。みかん
表示順序を入れてください。:閲覧数の多い順
反映先のスプレッドシートの名前を入れてください。みかん
検索結果数: 97044
閲覧数: 1,256,844
Q: 今まで一番びっくりしたプレゼントを教えてください！
A: ２０代前半の誕生日でした・・・。

帰宅したら玄関前になにやらみかん箱ほどの箱が・・・。

友人の名前が書いてあったのでわくわくして持ってはいろうとすると

なんか箱のなかでゴソゴソ・・・。

恐る恐る開封したらなんと

「イグアナ」でした、生きている！！

以前ゴムのおもちゃのイグアナをかわいい♪といったのを思い出しました＞＜

ゴムのおもちゃは可愛いけど生きているのはイヤだといって持ってかえってもらいました＞＜

彼女は本当は自分が欲しかったらしく

「こんなにかわいいのにねぇ・・・。○ちゃん（私）はあんたがきらいなんやって＠－＠よちよち」

と・・・・・・＞ｍ＜。。。

こんなことになりそうな気がしたから♪といってゴムのイグアナをくれました・・・。

もう今更ゴムのイグアナもいらない＞＜と思いましたが・・・・。

車にいれておいたら道行く人が驚いていました・・・。

あれから１５年以上たちましたが

あんなサプライズプレゼントはありませんね＞＜。。

その次の年はインコをくれました。

こども達が生まれたら出目金やらベタをつれてきました。

おかげで生き物と一緒に大きくなりました。

情操教育に、と言っていましたが・・・ホントは自分が欲しいものをくれたのだと思います。。。
-----------------
閲覧数: 692,581
Q: 娘が、家で 女の子やおじさんの姿を見る…と言います。 3人の娘の母です。 小3の長女が、昨年夏頃より「教室の自分の机の引き出しから光の玉が出てきて、目の前で消えた」 とか「学校のトイ
A: 僕は、娘さんの立場です。

僕も普段から霊体験をしているのですが、その事を両親に話しても、信じてくれないどころか、気がおかしいのか？と言われてしまいます。

あと、そういう話をすると、その場の空気を壊しますし、「お前のせいで、夜、トイレに行けなくなってしまったじゃんか」と怒られたり、毎月お墓参りに